# Example for Testing Storm-Chaser

This notebook generates some simple data in order to test that the StormChaser is working properly

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

## Create Data

#### Define Data Function

In [ ]:
def quadratic_func(x, a, b, c):
    return a * (x**2) + b * x + c


def generate_data(
    size,
    seed=None,
    x_range=[0, 10],
    a_range=[0.8, 1.2],
    b_range=[0, 2],
    c_range=[0.5, 1.5],
    d_range=[-1, 1],
    func=quadratic_func,
):
    rng = np.random.RandomState(seed)

    # generate x values
    x = rng.uniform(*x_range, size=size)

    # generate parameters
    a = rng.uniform(*a_range, size=size)
    b = rng.uniform(*b_range, size=size)
    c = rng.uniform(*c_range, size=size)
    d = rng.uniform(*d_range, size=size)

    # compute y values
    weight = func(x, a, b, c)

    df = pd.DataFrame(
        {
            "x": x,
            "a": a,
            "b": b,
            "c": c,
            "d": d,
            "weight": weight,
        }
    )

    return df

#### Generate Data Set

In [ ]:
cfg = {
    "size": 1000000,
    "seed": 42,
    "x_range": [1, 10],
    "a_range": [0.8, 1.2],
    "b_range": [0, 2],
    "c_range": [0.5, 1.5],
    "d_range": [-1, 1],
}

df = generate_data(**cfg)

cfg_base = {
    "size": cfg["size"],
    "seed": 43,
    "x_range": cfg["x_range"],
    "a_range": [np.mean(cfg["a_range"]), np.mean(cfg["a_range"])],
    "b_range": [np.mean(cfg["b_range"]), np.mean(cfg["b_range"])],
    "c_range": [np.mean(cfg["c_range"]), np.mean(cfg["c_range"])],
    "d_range": [np.mean(cfg["d_range"]), np.mean(cfg["d_range"])],
}
df_base = generate_data(**cfg_base)

#### Plot Variations

In [ ]:
fig, ax = plt.subplots()
x = np.linspace(*cfg["x_range"], 100)
mask = df["a"] > 1.18
ax.scatter(df["x"][mask], df["weight"][mask])
mask = df["a"] < 0.82
ax.scatter(df["x"][mask], df["weight"][mask])
ax.plot(x, quadratic_func(x, 1.19, 1, 1), color="black")
ax.plot(x, quadratic_func(x, 1, 1, 1), color="black")
ax.plot(x, quadratic_func(x, 0.81, 1, 1), color="black")
ax.set_xlabel("x")
ax.set_ylabel("weight")

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df["x"], df["weight"])
ax.set_xlabel("x")
ax.set_ylabel("weight")

## Create StormChaser

#### Create Model

In [ ]:
from storm_chaser.storm import StormChaser


chaser = StormChaser(
    params=["x"],
    params_sys=[
        "a",
        "b",
        "c",
        "d",
    ],
    param_bounds={
        "x": cfg["x_range"],
        "a": cfg["a_range"],
        "b": cfg["b_range"],
        "c": cfg["c_range"],
        "d": cfg["d_range"],
    },
    data_trafo={},
    params_trafo_bin_width_range={
        # "x": [0.1, 1.0],
        "x": [0.05, 0.3],
    },
    min_target_per_bin=10000,
    min_events_per_bin=100,
    use_nth_fc_layer_as_input=None,
    verbose=True,
)

#### Train Model

In [ ]:
import tensorflow as tf

chaser.build_model(
    df_sys=df,
    max_n_sys=None,
    ignore_missing_samples=True,
    epochs=1,
    steps_per_epoch=20000,
    steps_burn_in=0,
    num_trafo_samples=1000,
    weight_key="weight",
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    bootstrap_mc=True,
)

## Validate Model

In [ ]:
from itertools import cycle


def get_color_cycler(colors=plt.rcParams["axes.prop_cycle"].by_key()["color"]):
    return cycle(colors)


def compare_model(
    chaser,
    params_sys,
    cfg,
    func=quadratic_func,
    plot_truth=True,
    fig=None,
    ax=None,
    label_truth="Truth",
    label_sys="Prediction",
    kwargs_truth={},
    kwargs_sys={},
):
    x = np.linspace(*cfg["x_range"], 20)

    y_base = func(
        x=x,
        a=np.mean(cfg["a_range"]),
        b=np.mean(cfg["b_range"]),
        c=np.mean(cfg["c_range"]),
    )
    y_sys = func(
        x=x,
        a=params_sys["a"],
        b=params_sys["b"],
        c=params_sys["c"],
    )

    variation = y_sys / y_base
    df_input = {"x": x}
    for param in chaser.params_sys:
        df_input[param] = np.zeros_like(x) + params_sys[param]
    df_input = pd.DataFrame(df_input)

    model_values = chaser.chase(df_input)
    model_pred = model_values[..., 0, 0]
    model_unc = model_values[..., 0, 1]

    if ax is None:
        fig, ax = plt.subplots()

    if plot_truth:
        ax.plot(x, variation, label=label_truth, **kwargs_truth)

    ax.plot(x, model_pred, label=label_sys, **kwargs_sys)
    ax.fill_between(
        x,
        model_pred - model_unc,
        model_pred + model_unc,
        alpha=0.5,
        **kwargs_sys,
    )
    ax.set_xlabel("x")
    ax.set_ylabel("Variation")
    ax.legend()
    # ax.set_ylim(0, 2)
    fig.tight_layout()

    return fig, ax


fig, ax = plt.subplots()
color_cycler = get_color_cycler()

for a in [1.2, 1.15, 1.0, 0.9, 0.85, 0.8]:
    color = next(color_cycler)
    compare_model(
        chaser,
        params_sys=dict(a=a, b=1, c=1, d=0),
        cfg=cfg,
        fig=fig,
        ax=ax,
        kwargs_truth=dict(ls="-", color=color),
        kwargs_sys=dict(ls="--", color=color),
        label_sys=None,
        label_truth=f"a = {a:2.2f}",
    )

#### Keep all other systematics at baseline

In [ ]:
from copy import deepcopy

for param in chaser.params_sys:
    if param in ["size", "seed", "x_range"]:
        continue

    fig, ax = plt.subplots()
    ax.set_title(f"Varyinng: {param}")
    color_cycler = get_color_cycler()

    bounds = cfg[f"{param}_range"]
    for value in np.linspace(*bounds, 7)[1:-1]:
        color = next(color_cycler)
        params_sys = dict(
            a=np.mean(cfg["a_range"]),
            b=np.mean(cfg["b_range"]),
            c=np.mean(cfg["c_range"]),
            d=np.mean(cfg["d_range"]),
        )
        params_sys[param] = value

        compare_model(
            chaser,
            params_sys=params_sys,
            cfg=cfg,
            fig=fig,
            ax=ax,
            kwargs_truth=dict(ls="-", color=color),
            kwargs_sys=dict(ls="--", color=color),
            label_sys=None,
            label_truth=f"{param} = {value:2.2f}",
        )
    ax.plot(np.inf, np.inf, color="0.3", ls="-", label="Truth")
    ax.plot(np.inf, np.inf, color="0.3", ls="--", label="Prediction")
    ax.legend()

#### Randomly choose other parameters

In [ ]:
from copy import deepcopy

for param in chaser.params_sys:
    if param in ["size", "seed", "x_range"]:
        continue

    fig, ax = plt.subplots()
    ax.set_title(f"Varyinng: {param}")
    color_cycler = get_color_cycler()

    bounds = cfg[f"{param}_range"]
    for value in np.linspace(*bounds, 7)[1:-1]:
        color = next(color_cycler)
        params_sys = dict(
            a=np.random.uniform(*cfg["a_range"]),
            b=np.random.uniform(*cfg["b_range"]),
            c=np.random.uniform(*cfg["c_range"]),
            d=np.random.uniform(*cfg["d_range"]),
        )
        params_sys[param] = value

        label_truth = ""
        for name, val in params_sys.items():
            label_truth += f"{name}: {val:2.2f} | "
        compare_model(
            chaser,
            params_sys=params_sys,
            cfg=cfg,
            fig=fig,
            ax=ax,
            kwargs_truth=dict(ls="-", color=color),
            kwargs_sys=dict(ls="--", color=color),
            label_sys=None,
            label_truth=label_truth,
        )
    ax.plot(np.inf, np.inf, color="0.3", ls="-", label="Truth")
    ax.plot(np.inf, np.inf, color="0.3", ls="--", label="Prediction")
    ax.legend()